In [1]:
### Function that takes in a dataframe, removes outliers, problably normalize and scale data
# Functions are set up in such a way where we should be able to batch pre-process the data without reading in the entire data set at once

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Set types
# https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude/8674#8674
# Floats have 7 decimal digits of precision, doubles 15. You only need around 7 for lat and long 
types_dict = {
        "fare_amount": np.float16,
        "pickup_longitude": np.float32, 
        "pickup_latitude": np.float32, 
        "dropoff_longitude": np.float32, 
        "dropoff_latitude": np.float32, 
        "passenger_count": np.int8,
        "pickup_datetime": np.int8,
}

In [4]:
def _remove_outliers(df):
    """Takes in an unprocessed df and returns a df that has outliers removed"""
    ## Remove obvious outliers first
    # where lat/long == 0
    df = df.drop(df.index[df['pickup_longitude'] == 0])
    df = df.drop(df.index[df['dropoff_longitude'] == 0])
    # Where fares < 0, becuase that makes no sense
    df = df.drop(df.index[df['fare_amount'] < 0])
    # Realistically, if we want to keep the trips within manhattan, pickup long should be between -72 and -75 and lat between 40 and 42
    # as everything else is an extreme outlier
    df = df.drop(df.index[df['pickup_longitude'] > -72])
    df = df.drop(df.index[df['pickup_longitude'] < -75])
    df = df.drop(df.index[df['pickup_latitude'] > 42])
    df = df.drop(df.index[df['pickup_latitude'] < 40])
    # Visualizations show the same thing with dropof longs and lats
    df = df.drop(df.index[df['dropoff_longitude'] > -72])
    df = df.drop(df.index[df['dropoff_longitude'] < -75])
    df = df.drop(df.index[df['dropoff_latitude'] > 42])
    df = df.drop(df.index[df['dropoff_latitude'] < 40])
    return df

# we're assuming that a df of the right types is being passed in
def preprocess(df,types=types_dict):
    """Takes in an unprocessed df and returns a df that has outliers removed and is preprocessed"""
    # Drop NA values
#     df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
#     df = df.drop()
    # Remove obvious outliers first
    df = _remove_outliers(df)
    # Key isnt useful
    del df["key"]
    # Currently giving categorial numbers, but something like onehotencoding might work better
    df["pickup_datetime"] = df["pickup_datetime"].apply(lambda d:int(d[10:13]))
    # Set types as we batch read in
    for name, t in types_dict.items():
        df[name] = df[name].astype(t)
    # TODO - Scale data. Requires us to find min and max of entire dataset, not sure how to proceed since we're batching
    # TODO - Normalize data, same issues as above and batching.
    # TODO - Possibly PCA, but seems like all of the features we get should be relevant to the task at hand
    return df

In [5]:
## Load clean data in to dataframe

chunksize = 10 ** 6
filename = "../data/train.csv"

dataset_df = pd.DataFrame()  # main dataset

from tqdm import tqdm
# Pretty progress bar
pbar = tqdm(total=len(range(54)))
for index, df_chunk in enumerate(pd.read_csv(filename, chunksize=chunksize)):
    pbar.update(1)
    dataset_df = pd.concat([dataset_df, preprocess(df_chunk)])
pbar.close()

size_bytes = dataset_df.memory_usage(deep=True).sum()
print("Entire dataset fits in {:.2f} GB".format(size_bytes/1e+9))
print("Number of rows: {}".format(dataset_df.count()[0]))

56it [08:58, 12.64s/it]


Entire dataset fits in 1.52 GB
Number of rows: 54255094


## Data Loaded

In [6]:
!pip install sklearn

In [8]:
# from scipy.cluster.vq import vq, kmeans, whiten

In [12]:
# kmeans()

In [13]:
# import sklearn

In [9]:
from sklearn.cluster import KMeans

In [18]:
KMeans

sklearn.cluster.k_means_.KMeans

In [21]:
# kmeans = KMeans(n_clusters=2, random_state=0).fit(dataset_df)

In [22]:
# print("test")

test


In [14]:
data = dataset_df.iloc[:100000]

In [15]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(data)

In [16]:
kmeans.predict(dataset_df.iloc[:100000], sample_weight=None)

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)